# VacationPy
----


In [1]:
# Dependencies and Setup
mport pandas as pd
import requests
import gmaps
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame

In [3]:
# Load the csv exported in WeatherPy to a DataFrame
weatherdata = pd.read_csv("Output/weather_data.csv")
weatherdata.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,cape town,75,ZA,1579998942,88,-33.93,18.42,64.00,11.41
1,taraclia,78,MD,1579998942,44,45.90,28.67,34.95,3.42
2,rikitea,18,PF,1579998882,77,-23.12,-134.97,79.25,14.03
3,busselton,19,AU,1579998920,37,-33.65,115.33,81.00,8.79
4,arlit,16,NE,1579998943,10,18.74,7.39,66.99,8.59


### Humidity Heatmap

In [47]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations and Humidity as the weight.
locations = weatherdata[["Lat", "Lng"]]
humidity = weatherdata["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure(map_type="TERRAIN")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=weatherdata["Humidity"].max(),
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions:
** A max temperature lower than 80 degrees but higher than 70.
** Wind speed less than 10 mph.
** Zero cloudiness.
* Drop rows with null values.

In [30]:
weatherdata[["Max Temp","Wind Speed","Cloudiness"]].astype(float)
ideal_weather = weatherdata.loc[(weatherdata["Max Temp"] < 80)]
ideal_weather = weatherdata.loc[(weatherdata["Max Temp"] > 70)]
ideal_weather = weatherdata.loc[(weatherdata["Wind Speed"] < 10)]
ideal_weather = weatherdata.loc[(weatherdata["Cloudiness"] == 0)]
hotel_df = ideal_weather.dropna().set_index("City").reset_index()
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,aswan,0,EG,1579998944,54,24.09,32.91,53.60,8.05
1,lebu,0,CL,1579998943,69,-37.62,-73.65,64.08,10.94
2,abu samrah,0,SY,1579998946,56,35.30,37.18,38.28,8.23
3,kirakira,0,SB,1579998946,66,-10.45,161.92,85.98,5.01
4,port alfred,0,ZA,1579998947,98,-33.59,26.89,66.99,7.00


### Hotel Map
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [31]:
hotel_df["Hotel Name"] = ""

In [56]:
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key
}
for index,row in hotel_df.iterrows():
    # get lat, lng from df
    lng = row["Lng"]
    lat = row["Lat"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
#     print the hotel_name url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    hotel_name = hotel_name.json()
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        pass
        
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,aswan,0,EG,1579998944,54,24.09,32.91,53.60,8.05,Hotel Sofitel Legend Old Cataract Aswan
1,lebu,0,CL,1579998943,69,-37.62,-73.65,64.08,10.94,Hotel Plaza Lebu
2,abu samrah,0,SY,1579998946,56,35.30,37.18,38.28,8.23,
3,kirakira,0,SB,1579998946,66,-10.45,161.92,85.98,5.01,
4,port alfred,0,ZA,1579998947,98,-33.59,26.89,66.99,7.00,The Halyards Hotel


In [68]:
# Remove rows with empty cells
hotel_df = hotel_df.loc[(hotel_df["Hotel Name"] != "")]
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,aswan,0,EG,1579998944,54,24.09,32.91,53.60,8.05,Hotel Sofitel Legend Old Cataract Aswan
1,lebu,0,CL,1579998943,69,-37.62,-73.65,64.08,10.94,Hotel Plaza Lebu
4,port alfred,0,ZA,1579998947,98,-33.59,26.89,66.99,7.00,The Halyards Hotel
5,qaanaaq,0,GL,1579998947,44,77.48,-69.36,-12.51,5.12,Qaanaaq Hotel
6,tongliao,0,CN,1579998948,77,43.61,122.27,2.98,4.07,Country Garden Phoenix Hotel Tongliao


In [63]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [67]:
# Add marker layer ontop of heat map
fig = gmaps.figure(map_type = "TERRAIN")
markers = gmaps.marker_layer(
    locations,
    info_box_content = hotel_info
)
fig.add_layer(markers)
fig.add_layer(heat_layer)
fig
# Display Map

Figure(layout=FigureLayout(height='420px'))

## Most of the hotels with my ideal vacation conditions are in Asia